# Rewards Analytics and Distribution Dashboard
This goal of this notebook is to offer an easy way to process the outputs of the praise and sourcecred reward systems, perform an analysis of the results and calculate the token reward distribution. It uses mock data and should be considered a work-in-progress. 

### Imports
First, we import the relevant libraries, get the Data and set how many tokens we want to distribute

In [ ]:
from ipyfilechooser import FileChooser

import os
import sys
import inspect

import pandas as pd 
import numpy as np 
import analytics_toolbox as tb

import holoviews as hv
from holoviews import opts
import plotly.graph_objects as go
import plotly.express as px

import base64
from IPython.display import HTML

fc_praise = FileChooser('./')
fc_sourcecred = FileChooser('./')
fc_rewardboard = FileChooser('./')

print("== Please choose the Praise CSV file == ")
display(fc_praise)
print("== Please choose the Sourcecred CSV file == ")
display(fc_sourcecred)
print("== Please choose the Rewardboard address list CSV file == ")
display(fc_rewardboard)

Now that we have selected the files, we can import them for processing. We can also set the total amount of tokens we want to distribute this period. We also set the name we want our output file to have.

Tip: Now that the file paths are set, you can safely click on "Cell > Run all below" from here on the menu bar to execute everything :) 

### Parameters

Set the total amount of tokens we want to distribute this period. We also set the name we want our output file to have.

In [ ]:

NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE = 1950
#NUMBER_OF_SOURCECRED_REWARD_TOKENS_TO_DISTRIBUTE = 1950
NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS = 500
NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD = 100
NUMBER_OF_QUANTIFIERES_PER_PRAISE = 3

OUTPUT_FILENAME = "rewards_round_01"

In [ ]:
PRAISE_DATA_PATH = fc_praise.selected
SOURCECRED_DATA_PATH = fc_sourcecred.selected
REWARD_BOARD_ADDRESSES_PATH = fc_rewardboard.selected

praise_data = pd.read_csv(PRAISE_DATA_PATH)
sourcecred_data = pd.read_csv(SOURCECRED_DATA_PATH)
rewardboard_addresses = pd.read_csv(REWARD_BOARD_ADDRESSES_PATH)

In [ ]:
hv.extension('bokeh')

## Reward allocation

### Praise

This method allocates the praise rewards in a very straightforward way: It adds the value of all dished praised together, and then assigns to each user their % of the total.

In [ ]:
def calc_praise_rewards(praiseData, tokensToDistribute):
    #we discard all we don't need and and calculate the % worth of each praise
    #slimData = praiseData[['FROM USER ACCOUNT', 'TO USER ACCOUNT', 'FINAL QUANT']].copy()
    totalPraisePoints = praiseData['FINAL QUANT'].sum()

    praiseData['PERCENTAGE'] = praiseData['FINAL QUANT']/totalPraisePoints
    praiseData['TOKEN TO RECEIVE'] = praiseData['PERCENTAGE'] * tokensToDistribute
    return praiseData

praise_distribution = calc_praise_rewards(praise_data.copy(), NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE)
praise_distribution.style


### SourceCred
For now Sourcecred does the distribution independently, but if this changed we would calculate the rewards in the same way.

In [ ]:
def calc_sourcecred_rewards(sourcecredData, tokensToDistribute):
    #we discard all we don't need and and calculate the % worth of each praise
    slimData = sourcecredData[['IDENTITY', 'AMOUNT']].copy()
    totalGrainPoints = slimData['AMOUNT'].sum()

    slimData['PERCENTAGE'] = slimData['AMOUNT']/totalGrainPoints
    slimData['TOKEN TO RECEIVE'] = slimData['PERCENTAGE'] * tokensToDistribute
    return slimData

#sourcecred_distribution = calc_sourcecred_rewards(sourcecred_data.copy(), NUMBER_OF_SOURCECRED_REWARD_TOKENS_TO_DISTRIBUTE)
sourcecred_distribution = sourcecred_data.copy()

## Preparing and combining the Datasets

Now that we have both distributions, we can combine them into one table.
But before that, we need to prepare the data and clean it a bit. We also use the chance to generate a table which shows us how much praise each user received. We'll use it later in our analysis.

In [ ]:

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -removes the '#' and following from discord names
#  -Some renaming and dropping 
def prepare_praise(praise_data):

    #praise_data['TO USER ACCOUNT'] = (praise_data['TO USER ACCOUNT'].str.split('#', 1, expand=False).str[0]).str.lower()
    
    praise_data.rename(columns = {'TO USER ACCOUNT':'USER IDENTITY'}, inplace = True)
    praise_data.rename(columns = {'TO ETH ADDRESS':'USER ADDRESS'}, inplace = True)
    praise_data['USER ADDRESS'].fillna('MISSING USER ADDRESS', inplace=True)
    
    processed_praise = praise_data[['USER IDENTITY', 'USER ADDRESS', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    praise_by_user = praise_data[['USER IDENTITY', 'USER ADDRESS', 'FINAL QUANT', 'PERCENTAGE', 'TOKEN TO RECEIVE']].copy().groupby(['USER IDENTITY', 'USER ADDRESS']).agg('sum').reset_index()
    
    return processed_praise, praise_by_user

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -Some renaming and dropping 
#  -changing percentages from 0 - 100 to 0.00-1.00
def prepare_sourcecred(sourcecred_data):
    
    sourcecred_data.rename(columns = {'%':'PERCENTAGE SOURCECRED'}, inplace = True)
    sourcecred_data.rename(columns = {'IDENTITY' : 'USER ADDRESS'}, inplace = True)
    
    sourcecred_data = sourcecred_data[['USER ADDRESS', 'PERCENTAGE SOURCECRED', 'TOKEN TO RECEIVE']]
    
    return sourcecred_data


processed_praise, praise_by_user = prepare_praise(praise_distribution.copy())
processed_sourcecred = prepare_sourcecred(sourcecred_distribution.copy())
praise_by_user.style


Let's also create a table which will let us focus on the quantifiers. It will show us what value each quantifier gave to each single praise item.

In [ ]:
def data_by_quantifier(praise_data):
    quant_only = pd.DataFrame()
    praise_data.drop(['DATE', 'TO USER ACCOUNT', 'TO ETH ADDRESS', 'FROM USER ACCOUNT', 'FROM ETH ADDRESS', 'REASON', 'SOURCE ID', 'SOURCE NAME', 'AVG SCORE', 'CORRECTION ADD', 'CORRECTION SUB', 'CORRECTION COMMENT','FINAL QUANT'], axis=1, inplace=True)
    num_of_quants = int((praise_data.shape[1] -1)/ 4)
    for i in range(num_of_quants):
        q_name =  str( 'QUANTIFIER '+ str(i+1) +' USERNAME' )
        q_addr =  str( 'QUANTIFIER '+ str(i+1) +' ETH ADDRESS')
        q_value = str('SCORE '+str(i+1) )
        buf = praise_data[['ID', q_name , q_addr, q_value ]].copy()
    
        buf.rename(columns={q_name: 'QUANT_ID', q_addr: 'QUANT_ADDRESS', q_value: 'QUANT_VALUE', 'ID':'PRAISE_ID'}, inplace=True)
        #print(buf)
        quant_only = quant_only.append(buf.copy(), ignore_index=True)

    columnsTitles = ['QUANT_ID', 'QUANT_ADDRESS', 'PRAISE_ID', 'QUANT_VALUE']
    quant_only.sort_values(['QUANT_ID', 'PRAISE_ID'], inplace=True)
    quant_only =  quant_only.reindex(columns=columnsTitles).reset_index(drop=True)
    return quant_only

quantifier_rating_table = data_by_quantifier(praise_data.copy())
quantifier_rating_table.style


Now, we will calculate the rewards for the Quantifiers and the Reward Board. This is fairly straightforward, since we distribute the allocated tokens equally.


In [ ]:
quantifier_rewards = pd.DataFrame(quantifier_rating_table[['QUANT_ID','QUANT_ADDRESS']].value_counts().reset_index().copy())
quantifier_rewards.rename(columns={ quantifier_rewards.columns[2]: "NUMBER_OF_PRAISES" }, inplace = True)
quantifier_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS / len(quantifier_rewards.index)

    
quantifier_rewards.style

In [ ]:
rewardboard_rewards = pd.DataFrame(rewardboard_addresses)
rewardboard_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD / len(rewardboard_rewards.index)
rewardboard_rewards.style

Now we can merge them all into one table and save it, ready for distribution!

In [ ]:
def prepare_total_data_chart(praise_rewards, sourcecred_rewards, quantifier_rewards, rewardboard_rewards):
    
    praise_rewards = praise_rewards.copy()[['USER IDENTITY', 'USER ADDRESS', 'TOKEN TO RECEIVE']].rename(columns = {'TOKEN TO RECEIVE':'PRAISE_REWARD'})
    praise_rewards['USER ADDRESS'] = praise_rewards['USER ADDRESS'].str.lower()
    
    sourcecred_rewards = sourcecred_rewards.copy()[['USER ADDRESS', 'TOKEN TO RECEIVE']].rename(columns = {'TOKEN TO RECEIVE':'SOURCECRED_REWARD'})
    sourcecred_rewards['USER ADDRESS'] = sourcecred_rewards['USER ADDRESS'].str.lower()
    
    quantifier_rewards.rename(columns = {'QUANT_ADDRESS':'USER ADDRESS', 'QUANT_ID': 'USER IDENTITY', 'TOKEN TO RECEIVE':'QUANT_REWARD'}, inplace = True)
    quantifier_rewards['USER ADDRESS'] = quantifier_rewards['USER ADDRESS'].str.lower()
    
    rewardboard_rewards.rename(columns = {'ID':'USER ADDRESS', 'TOKEN TO RECEIVE': 'REWARDBOARD_REWARD'}, inplace = True)
    rewardboard_rewards['USER ADDRESS'] = rewardboard_rewards['USER ADDRESS'].str.lower()
    
    
    final_allocations = pd.merge(rewardboard_rewards, quantifier_rewards , on=['USER ADDRESS','USER ADDRESS'], how='outer')
    final_allocations = pd.merge(final_allocations , sourcecred_rewards, on=['USER ADDRESS','USER ADDRESS'],how='outer')
    final_allocations = pd.merge(final_allocations, praise_rewards, left_on=['USER ADDRESS','USER IDENTITY'], right_on=['USER ADDRESS','USER IDENTITY'], how='outer')
    
    final_allocations['USER IDENTITY'].fillna('missing username', inplace = True)
    final_allocations.fillna(0, inplace = True)
    final_allocations['TOTAL TO RECEIVE'] = final_allocations['PRAISE_REWARD'] + final_allocations['SOURCECRED_REWARD'] + final_allocations['QUANT_REWARD'] + final_allocations['REWARDBOARD_REWARD']
    
    final_allocations = final_allocations.sort_values(by= 'TOTAL TO RECEIVE', ascending  = False).reset_index(drop=True)
    return final_allocations

final_token_allocations = prepare_total_data_chart(praise_by_user.copy(), processed_sourcecred.copy(), quantifier_rewards.copy(), rewardboard_rewards.copy())
final_token_allocations.style

#TODO: FIX COLUMN ORDER


# Results Analysis
Let's dive into some data analysis! We'll use the metrics designed and explained by octopus🐙, and focus only on the praise rewards for now. Starting with:


### Allocation percentages
This table will show us which percentage if the total rewards gets distributed to which top % of users. So "Top 50% -> 0.85" would mean that the top 50% of praisees received 85% of the total rewards 

In [ ]:
p_vals = np.array([50,80,90,95,99])
rewards_rp = np.array([tb.resource_percentage(praise_by_user["TOKEN TO RECEIVE"], p) for p in p_vals])

my_rd_index = [("Top " + str(100 - p) +"%") for p in p_vals]
resource_distribution = pd.DataFrame({"Rewards": rewards_rp}, index = my_rd_index)
resource_distribution

### Gini coefficient
Next we will look at the Gini coefficient. Note that there is some debate if we want to use this metric at all, since it is usually employed to measure wealth distribution, and not compensation.

In [ ]:
p_vals = np.array([0, 50, 80])
rewards_gc = np.array([tb.gini_gt_p(np.array(praise_by_user["TOKEN TO RECEIVE"]), p) for p in p_vals])

my_index = ["All", "Top 50%", "Top 20%"]
gini_coefs = pd.DataFrame({"Rewards": rewards_gc}, index = my_index)
gini_coefs

### Shannon Entropy

[Shannon Entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) is a concept from communications theory, which is also used in measuring the diversity of a distribution. The formula for calculating Shannon Entropy among $n$ individuals is
    $$\\sum_{k=1}^n -p_k log_2(p_k),$$
where $p_k$ represents the proportion of the resource that user $k$ received.

Here we compare the actual Shannon Entropy with the maximum possible for the dataset, keeping in mind that a Shannon Entropy of 0 would mean one user holds all the rewards


In [ ]:
entropies_df = pd.DataFrame(data = {"Rewards" : tb.calc_shannon_entropies(praise_by_user["PERCENTAGE"]) }, index = ["Entropy", "Max Entropy", "% of Max"])
entropies_df

### Nakamoto Coefficient
Last but not least, the Nakamoto coefficient. The Nakamato Coefficient is defined as the smallest number of accounts who control at least 50% of the resource. Although its significance relates to the prospect of a 51% attack on a network, which may not be relevant in our context, we can still use it as an intuitive measure of how many individuals received the majority of a resource.

In [ ]:
ak_coef_IH = tb.nakamoto_coeff(praise_by_user, "PERCENTAGE")
ak_coef_IH

## Praise Data Visualization

### Rating distribution
Since praise gets valued on a scale, we can take a look at how often each value of the scale gets assigned by quantifiers.

In [ ]:
freq = quantifier_rating_table[['QUANT_VALUE']].value_counts().rename_axis('QUANT_VALUE').reset_index(name='counts').sort_values(by=['QUANT_VALUE'])
freq['QUANT_VALUE'] = freq['QUANT_VALUE'].astype('string')

fig_freq = px.bar(freq, x="QUANT_VALUE", y="counts", labels={"QUANT_VALUE": "Rating","counts": "Number of appearances"}, title="Praise Rating Distribution", width=800, height=300)
fig_freq.show()


### Praise Reward Distribution

We can also take a look at the distribution of the received praise rewards

In [ ]:
pr_distribution = praise_by_user[['USER IDENTITY', 'PERCENTAGE']].sort_values(by=['PERCENTAGE'], ascending=False)

fig_pr_distribution = px.bar(pr_distribution, x='USER IDENTITY', y='PERCENTAGE', labels={"IDENTITY": "User","PERCENTAGE": "% of total"}, title="Praise Reward Distribution")#.opts(width=800, height=500, title='SourceCred Distribution', xlabel='Value', ylabel='% of Total', xaxis='bare')
fig_pr_distribution.update_xaxes(showticklabels=False)

fig_pr_distribution.show()


### Praise Flows

Now for something more fun: let's surface the top "praise flows" from the data. Thanks to @inventandchill for this awesome visualization! 
On one side we have the top 20 praise givers separately (modifiable by changing the variable n_senders), on the other the top 25 receivers (modifiable by changing the variable n_receivers). The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories.

In [ ]:
NUMBER_OF_SENDERS_FLOW = 15 #The left side, the praise senders. X largest ones + one bucket for the rest 
NUMBER_OF_RECEIVERS_FLOW = 25 #The right side, the praise receivers. X largest ones + one bucket for the rest 
dist_for_praise_flow = praise_distribution.rename(columns = {'FROM USER ACCOUNT':'FROM', 'TO USER ACCOUNT':'TO'})
praise_flow = tb.prepare_praise_flow(dist_for_praise_flow.copy(), n_senders=NUMBER_OF_SENDERS_FLOW, n_receivers=NUMBER_OF_RECEIVERS_FLOW)
#praise_flow

In [ ]:
%%opts Sankey (cmap='Category10' edge_color='FROM' edge_line_width=0 node_alpha=1.0)
%%opts Sankey [node_sort=False label_position='outer' bgcolor="snow" node_width=40 node_sort=True ]
%%opts Sankey [width=1000 height=800 title="Praise flow for Batch 1. Sum of Praise. Left - praise sender. Right - praise receiver"]
%%opts Sankey [margin=0 padding=0 show_values=True]

hv.Sankey(praise_flow, kdims=["FROM", "TO"], vdims=["FINAL QUANT"])

## SourceCred Data

### SourceCred token Distribution

Next we can see the distribution made by the SourceCred algorithm.

In [ ]:

sc_distribution = processed_sourcecred[['USER ADDRESS', 'PERCENTAGE SOURCECRED']].sort_values(by=['PERCENTAGE SOURCECRED'], ascending=False)

fig_sc_distribution = px.bar(sc_distribution, x='USER ADDRESS', y='PERCENTAGE SOURCECRED', labels={"USER ADDRESS": "User","PERCENTAGE SOURCECRED": "% of total"}, title="SourceCred Distribution")
fig_sc_distribution.update_xaxes(showticklabels=False)

fig_sc_distribution.show()


## Quantifier Data
Let's take a closer look at each quantifier. In the following step we will use the raw praise data to zoom in on how each quantifier scored the praises:

### Amount of praise quantified
With the above table we can easily see how much praise each quantifier rated.

In [ ]:
fig_pie = px.pie(quantifier_rewards, values='NUMBER_OF_PRAISES', names='QUANT_ID', title='Amount of praise per quantifier', labels={'NUMBER_OF_PRAISES': 'Praises quant'})
fig_pie.update_layout(showlegend=False)
fig_pie.show()

## Total Token Distribution Visualization and  Export
To send the allocations to the DAO for distribution, we need to put all data together and add the rewards for the reward board and the quantifiers. 

Let's take a final look at the total distribution. Click on the legend to filter out specific reward sources

In [ ]:
fig_final_alloc = px.bar(final_token_allocations, x="USER IDENTITY", y = ["QUANT_REWARD", "REWARDBOARD_REWARD", "PRAISE_REWARD", "SOURCECRED_REWARD"], title="Rewards received by category", color_discrete_map = {'PRAISE_REWARD': 'blue', 'SOURCECRED_REWARD': 'red', 'QUANT_REWARD':'green', 'REWARDBOARD_REWARD':'yellow'})
fig_final_alloc.update_xaxes(showticklabels=False)
fig_final_alloc.show()


That's it! We can now download the final distribution files and upload them to GitHub for future reference. We want to export 3 different files: <br>
   &emsp;- The final reward allocations, separated by source<br>
    &emsp;- The final reward allocations, in a disperse.app-readable format<br>
   &emsp;- The extended praise data, detailing the how many tokens each single praise netted (for future reference)<br>

In [ ]:
def create_download_link(csv, filename, download_title):
    filename = filename
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=download_title,filename=filename)
    return HTML(html)

final_allocation_csv = final_token_allocations.to_csv(index=False)
create_download_link(csv = final_allocation_csv, filename=OUTPUT_FILENAME, download_title = "Download final reward CSV file")

In [ ]:
final_alloc_disperse = final_token_allocations[['USER ADDRESS', 'TOTAL TO RECEIVE']].to_csv(sep=' ', index=False, header=False)
create_download_link(final_alloc_disperse, filename=(OUTPUT_FILENAME+"-disperse"), download_title = "Download disperse.app compatible file")

In [ ]:
praise_reward_export = praise_distribution.to_csv(index=False)
create_download_link(praise_reward_export, filename=OUTPUT_FILENAME+"-extendedPraise", download_title = "Download complete praise data")